# On the Effectiveness of Transfer Learning for Code Search

The following _Python Jupyter Notebook_ can be used to interactively resproduce the analysis we performed in our paper with the title *"On the Effectiveness of Transfer Learning for Code Search."*

### Initialization

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import re

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display
import numpy as np
import pandas as pd
import plotnine as p9
import scikit_posthocs as sp
from scipy import stats

from data_utils import *
from stats_utils import *

### Configuration

First, we configure some parameters for the script.

In [ ]:
from experiments_order import *
from experiments_groups import *
from experiments_rqs import *

TEST_VALUES_ORDER = ['js', 'ja', 'py', 'tp']
RQS_ORDER = ['rq1', 'rq2', 'rq3', 'rq4', 'rq5']
SIZES_ORDER = ['1k', 'full']
TYPES_ORDER = ['random', 'lucene', 'deepcs', 'zeroshot', 'nopretrain', 'finetuned', 'combined']
METRICS_ORDER = ['mrr', 'aroma', 'top_1', 'top_3', 'top_5', 'top_10']

DATA_CSV_DIRECTORY_PATH = 'resources/data'
AROMA_CSV_DIRECTORY_PATH = 'resources/aroma/data'

DATA_NN_CSV_FILE_NAME_REGEX = re.compile(r'mem-\{(.+?)_(.+?)\}-\[(.+?)\]-\((.+?)\)')
DATA_ES_CSV_FILE_NAME_REGEX = re.compile(r'es-\((.+?)\)')
DATA_LU_CSV_FILE_NAME_REGEX = re.compile(r'lu-\((.+?)\)')
DATA_DC_CSV_FILE_NAME_REGEX = re.compile(r'dc-\[(.+?)\]-\((.+?)\)')
DATA_CO_CSV_FILE_NAME_REGEX = re.compile(r'lumem-\{(.+?)_(.+?)\}-\[(.+?)\]-\((.+?)\)')

CROSS_VALIDATION_FOLDS = 10

SIGNIFICANCE_LEVEL = 0.001

In [ ]:
from conf_labels import *
from conf_palettes import *

def experiment_label_renamer(label):
    result = label.upper()
    result = result.replace('-', 'ΓÇô')
    result = result.replace('_', '+')
    return result

## Study

We first define the functions to computer the metrics.

In [ ]:
def mrr(values, k=None):
    ranks = np.array(values)
    if k:
        ranks[ranks > k] = k + 1
    reciprocal_ranks = 1.0 / ranks
    result = np.mean(reciprocal_ranks)
    return result

In [ ]:
def top_k_acc(values, k=3):
    ranks = np.array(values)
    top_k = np.sum(ranks <= k)
    result = top_k / len(ranks)
    return result

Then, we read the data from the `CSV` files.

In [ ]:
df = pd.DataFrame()

# Iterate all the experiments files.
for experiment in EXPERIMENTS_ORDER:
    # Parse the labels.
    exp_query_bert, exp_code_bert, exp_train, exp_test = None, None, None, None
    if experiment in LUCENE_EXPERIMENTS:
        exp_test = DATA_LU_CSV_FILE_NAME_REGEX.findall(experiment)[0]
    elif experiment in DEEPCS_EXPERIMENTS:
        exp_train, exp_test = DATA_DC_CSV_FILE_NAME_REGEX.findall(experiment)[0]
    elif experiment in COMBINED_EXPERIMENTS:
        exp_query_bert, exp_code_bert, exp_train, exp_test = DATA_CO_CSV_FILE_NAME_REGEX.findall(experiment)[0]
    else:
        exp_query_bert, exp_code_bert, exp_train, exp_test = DATA_NN_CSV_FILE_NAME_REGEX.findall(experiment)[0]

    # Recognize the groups.
    if experiment in RANDOM_EXPERIMENTS:
        exp_type = 'random'
    elif experiment in LUCENE_EXPERIMENTS:
        exp_type = 'lucene'
    elif experiment in DEEPCS_EXPERIMENTS:
        exp_type = 'deepcs'
    elif experiment in ZEROSHOT_EXPERIMENTS:
        exp_type = 'zeroshot'
    elif experiment in NOPRETRAIN_EXPERIMENTS:
        exp_type = 'nopretrain'
    elif experiment in FINETUNED_EXPERIMENTS:
        exp_type = 'finetuned'
    elif experiment in COMBINED_EXPERIMENTS:
        exp_type = 'combined'
    
    # Recognize the RQ.
    if experiment in RQ1_EXPERIMENTS:
        exp_rq = 'rq1'
    elif experiment in RQ2_EXPERIMENTS:
        exp_rq = 'rq2'
    elif experiment in RQ3_EXPERIMENTS:
        exp_rq = 'rq3'
    elif experiment in RQ4_EXPERIMENTS:
        exp_rq = 'rq4'
    elif experiment in RQ5_EXPERIMENTS:
        exp_rq = 'rq5'

    # Load the CSV file.
    experiment_csv_file_path = os.path.join(DATA_CSV_DIRECTORY_PATH, experiment + '.csv')
    temp_df = pd.read_csv(experiment_csv_file_path, index_col=False)

    # Load the Aroma CSV file.
    aroma_experiment_csv_file_path = os.path.join(AROMA_CSV_DIRECTORY_PATH, experiment + '.csv')
    aroma_temp_df = pd.read_csv(aroma_experiment_csv_file_path, index_col=False)

    # Merge Aroma into the main data frame.
    temp_df = temp_df.merge(aroma_temp_df, on=['size', 'fold', 'stackoverflow_question_id', 'stackoverflow_answer_id'])

    # Compute the aggregation metrics.
    metrics_df = temp_df.groupby(['size', 'fold'], as_index=False).aggregate(
        mrr=('predicted_rank', mrr),
        top_1=('predicted_rank', lambda x: top_k_acc(x, 1)),
        top_3=('predicted_rank', lambda x: top_k_acc(x, 3)),
        top_5=('predicted_rank', lambda x: top_k_acc(x, 5)),
        top_10=('predicted_rank', lambda x: top_k_acc(x, 10)),
        aroma=('aroma', 'mean'),
    )

    # Sets the values.
    metrics_df['label'] = experiment
    metrics_df['query_bert'] = exp_query_bert
    metrics_df['code_bert'] = exp_code_bert
    metrics_df['train'] = exp_train
    metrics_df['test'] = exp_test
    metrics_df['type'] = exp_type
    metrics_df['rq'] = exp_rq
    
    # Add the partial dataframe to the main one.
    df = pd.concat([df, metrics_df], ignore_index=True)

# Transform the columns to categorical for easy sorting.
df['size'] = pd.Categorical(df['size'], categories=SIZES_ORDER)
df['label'] = pd.Categorical(df['label'], categories=reversed(EXPERIMENTS_ORDER))
df['test'] = pd.Categorical(df['test'], categories=TEST_VALUES_ORDER)
df['type'] = pd.Categorical(df['type'], categories=TYPES_ORDER)
df['rq'] = pd.Categorical(df['rq'], categories=RQS_ORDER)

display(df)

### Normality test

We test if the distributions for the experiments are normal.
To do that, we apply the *Shapiro-Wilk* test.
We cannot use the *DΓÇÖAgostino's K^2 Test*, since we have a limited number of observations (`10`) per experiment.

> *Null hypothesis*: the observations come from a normal distribution.
>
> `p-value >= 0.001`: accept the null hypothesis, normal (green)
>
> `p-value < 0.001`: reject the null hypothesis, not normal (red)

In [ ]:
# Create a dataframe with the p-values of the normality test.
normality_test_df = pivot_table_grouping(
    df,
    index=['test', 'rq', 'label', 'type'],
    columns='size',
    values=METRICS_ORDER,
    index_sort=[TEST_VALUES_ORDER, RQS_ORDER, EXPERIMENTS_ORDER, TYPES_ORDER],
    columns_sort=[METRICS_ORDER, SIZES_ORDER],
    aggfunc=lambda x: stats.shapiro(x)[1]
)

# Print the dataframe showing the acceptance of the null hypothesis as green, and reject as red.
display(normality_test_df
# Rename the values.
.rename(index=TEST_VALUES_LABELS, level=0)
.rename(index=RQS_LABELS, level=1)
.rename(index=experiment_label_renamer, level=2)
.rename(index=TYPES_LABELS, level=3)
.rename(columns=METRICS_LABELS, level=0)
.rename(columns=SIZES_LABELS, level=1)
# Show the p-values with reduced decimal digits.
.style.format('{:.4f}')
# # Apply the color filtering.
.applymap(lambda x: f'background-color: {RED_COLOR_HEX}' if x < SIGNIFICANCE_LEVEL else f'background-color: {GREEN_COLOR_HEX}')
)

Since there are many non-normal groups for the *full* size, we apply non-parametric tests and use the median as reference value.

### Median values comparison

We print the median values for all the models.

In [ ]:
median_values_df = pivot_table_grouping(
    df,
    index=['test', 'rq', 'label', 'type'],
    columns='size',
    values=METRICS_ORDER,
    index_sort=[TEST_VALUES_ORDER, RQS_ORDER, EXPERIMENTS_ORDER, TYPES_ORDER],
    columns_sort=[METRICS_ORDER, SIZES_ORDER],
    aggfunc=np.median,
)

# Print the dataframe showing the bars in the background.
display(median_values_df
# Rename the values.
.rename(index=TEST_VALUES_LABELS, level=0)
.rename(index=RQS_LABELS, level=1)
.rename(index=experiment_label_renamer, level=2)
.rename(index=TYPES_LABELS, level=3)
.rename(columns=METRICS_LABELS, level=0)
.rename(columns=SIZES_LABELS, level=1)
# Show the median values with reduced decimal digits.
.style.format('{:.4f}')
# Show a background bar as indication.
.bar(vmin=0.0, vmax=1.0, color=GREEN_COLOR_HEX)
)

To plot the data, we transform the dataframe from wide to long.

In [ ]:
# Transform data from wide to long.
long_df = pd.melt(df, id_vars=['label', 'size', 'fold', 'query_bert', 'code_bert', 'train', 'test', 'type', 'rq'], value_vars=METRICS_ORDER)

# Transform the variable column to categorical for easy sorting.
long_df['variable'] = pd.Categorical(long_df['variable'], categories=METRICS_ORDER)

display(long_df)

To plot the summary boxplots, we define a function.

In [ ]:
def plot_boxplot_summary(dataframe, size, title='', precision=0.05, mrr_limits=(0, 1), mrr_margin=0.1, aroma_limits=(0, 1), aroma_margin=0.1, figure_size=(14, 10)):
    # Subset the data.
    dataframe = dataframe.query('size == @size and variable in ["mrr", "aroma"]')

    # Preprocess the data for visualization.
    mrr_mask = dataframe['variable'] == 'mrr'
    aroma_mask = dataframe['variable'] == 'aroma'

    dataframe.loc[mrr_mask, 'tilemin'] = mrr_limits[0] - mrr_margin
    dataframe.loc[aroma_mask, 'tilemin'] = aroma_limits[0] + aroma_margin

    dataframe.loc[mrr_mask, ['ymin', 'ymax']] = mrr_limits[0] - mrr_margin, mrr_limits[1] + mrr_margin
    dataframe.loc[aroma_mask, ['ymin', 'ymax']] = aroma_limits[0] - aroma_margin, aroma_limits[1] + aroma_margin

    # Define the title.
    title = f'{title}\n\n' if title else ''

    # Return the plot.
    return (
        # Define the plot.
        p9.ggplot(dataframe)
        # Add a title.
        + p9.ggtitle(title=title)
        # Add the boxplots.
        + p9.geom_tile(p9.aes(x='label', y='tilemin', width=1.05, height=np.inf, fill='rq'), alpha=0.01)
        + p9.geom_boxplot(p9.aes(x='label', y='value', color='type'), position='dodge')
        # Fix the scales in grids.
        + p9.geom_blank(p9.aes(y='ymin'))
        + p9.geom_blank(p9.aes(y='ymax'))
        # Rename the y axis and change the ticks.
        + p9.scale_x_discrete(name='Experiment', labels=lambda l: [experiment_label_renamer(x) for x in l])
        # Rename the y axis and change the ticks.
        + p9.scale_y_continuous(name='Metric', breaks=np.linspace(0, 1, num=int(1 / precision) + 1), labels=lambda l: ['{:.2f}'.format(x) for x in l], expand=(0, 0))
        # Define the colors palettes.
        + p9.scale_color_manual(TYPES_PALETTE, name='Type', guide=False)
        + p9.scale_fill_manual(RQS_PALETTE, name='RQ', labels=lambda l: [RQS_LABELS[x] for x in l])
        # Place the plots in a wrap surface.
        + p9.facet_grid('test ~ variable', scales='free', labeller=p9.labeller(rows=lambda x: f'{TEST_VALUES_LABELS[x]}', cols=lambda x: f'{METRICS_LABELS[x]}'))
        # Set as horizontal.
        + p9.coord_flip()
        # Set up the legends.
        + p9.guides(fill=p9.guide_legend(override_aes={'alpha': 0.5}))
        # Define the theme for the plot.
        + p9.theme_bw()
        + p9.theme(
            # Remove the x and y axis name.
            axis_title_x=p9.element_blank(),
            axis_title_y=p9.element_blank(),
            # Set the size for the figure.
            figure_size=figure_size,
            # Place the legend on top, without title, and reduce the margin.
            legend_title=p9.element_blank(),
            legend_position='top',
            legend_margin=-10,
            legend_key_size=10,
        )
    )

#### 1K

In [ ]:
evalonek_boxplot = plot_boxplot_summary(
    long_df,
    size='1k',
    title='1K',
    precision=0.05,
    mrr_limits=(0, 0.45),
    mrr_margin=0.01,
    aroma_limits=(0.35, 0.65),
    aroma_margin=0.02,
    figure_size=(12, 20),
)
display(evalonek_boxplot)

#### Full

In [ ]:
evalfull_boxplot = plot_boxplot_summary(
    long_df,
    size='full',
    title='Full',
    precision=0.05,
    mrr_limits=(0, 0.25),
    mrr_margin=0.01,
    aroma_limits=(0.05, 0.5),
    aroma_margin=0.02,
    figure_size=(12, 30),
)
display(evalfull_boxplot)

To plot detailed boxplots, we define a function.

In [ ]:
def plot_boxplot_grid(dataframe, size, metric, precision=0.05, limits=(0, 1), x_margin=0.01, figure_size=(14, 10)):
    # Subset the data.
    dataframe = dataframe.query('size == @size and variable == @metric')
    
    # Remove unused categories.
    dataframe['type'] = dataframe['type'].cat.remove_unused_categories()

    # Return the plot.
    return (
        # Define the plot.
        p9.ggplot(dataframe)
        # Add the boxplots.
        + p9.geom_tile(p9.aes(x='label', y=limits[0], width=1.05, height=np.inf, fill='rq'), alpha=0.01)
        + p9.geom_boxplot(p9.aes(x='label', y='value', color='type'), position='dodge')
        # Rename the y axis and change the ticks.
        + p9.scale_x_discrete(name='Experiment', labels=lambda l: [experiment_label_renamer(x) for x in l])
        # Rename the y axis and change the ticks.
        + p9.scale_y_continuous(name='Metric', breaks=np.linspace(0, 1, num=int(1 / precision) + 1), limits=(limits[0] - x_margin, limits[1] + x_margin), labels=lambda l: ['{:.2f}'.format(x) for x in l], expand=(0, 0, 0, 0))
        # Define the colors palettes.
        + p9.scale_color_manual(TYPES_PALETTE, name='Type', guide=False)
        + p9.scale_fill_manual(RQS_PALETTE, name='RQ', labels=lambda l: [RQS_LABELS[x] for x in l])
        # Place the plots in a wrap surface.
        + p9.facet_wrap('test', scales='free', ncol=1, labeller=p9.labeller(cols=lambda x: f'{TEST_VALUES_LABELS[x]}'))
        # Set as horizontal.
        + p9.coord_flip()
        # Set up the legends.
        + p9.guides(fill=p9.guide_legend(override_aes={'alpha': 0.5}))
        # Define the theme for the plot.
        + p9.theme_bw()
        + p9.theme(
            # Remove the x and y axis name.
            axis_title_x=p9.element_blank(),
            axis_title_y=p9.element_blank(),
            # Add spacing between facets.
            subplots_adjust={'hspace': 0.2},
            # Set the size for the figure.
            figure_size=figure_size,
            # Place the legend on top, without title, and reduce the margin.
            legend_title=p9.element_blank(),
            legend_position='top',
            legend_margin=-10,
            legend_key_size=10,
        )
    )

#### MRR (1K)

In [ ]:
mrr_1k_boxplot = plot_boxplot_grid(
    long_df,
    size='1k',
    metric='mrr',
    precision=0.05,
    limits=(0, 0.45),
    x_margin=0.01,
    figure_size=(10, 20),
)
display(mrr_1k_boxplot)

#### MRR (Full)

In [ ]:
mrr_full_boxplot = plot_boxplot_grid(
    long_df,
    size='full',
    metric='mrr',
    precision=0.01,
    limits=(0, 0.22),
    x_margin=0.005,
    figure_size=(10, 35),
)
display(mrr_full_boxplot)

#### Top-10 (1K)

In [ ]:
top10_1k_boxplot = plot_boxplot_grid(
    long_df,
    size='1k',
    metric='top_10',
    precision=0.1,
    limits=(0, 0.7),
    x_margin=0.025,
    figure_size=(10, 20),
)
display(top10_1k_boxplot)

#### Top-10 (Full)

In [ ]:
top10_full_boxplot = plot_boxplot_grid(
    long_df,
    size='full',
    metric='top_10',
    precision=0.05,
    limits=(0, 0.45),
    x_margin=0.01,
    figure_size=(10, 35),
)
display(top10_full_boxplot)

#### Aroma (1K)

In [ ]:
aroma_1k_boxplot = plot_boxplot_grid(
    long_df,
    size='1k',
    metric='aroma',
    precision=0.1,
    limits=(0, 0.7),
    x_margin=0.01,
    figure_size=(10, 20),
)
display(aroma_1k_boxplot)

#### Aroma (Full)

In [ ]:
aroma_full_boxplot = plot_boxplot_grid(
    long_df,
    size='full',
    metric='aroma',
    precision=0.1,
    limits=(0, 0.5),
    x_margin=0.01,
    figure_size=(10, 35),
)
display(aroma_full_boxplot)

### Top-k trend

We study the trend of the *top-k* metric values when *k* increases.

First, we create a dataframe containing the mean values for the *top-k* metric values.

In [ ]:
# Select the top_k variable.
topk_long_df = long_df[long_df['variable'].isin(['top_1', 'top_3', 'top_5', 'top_10'])].copy()

# Transform the values from "top_k" to "k".
topk_long_df['variable'] = topk_long_df['variable'].apply(lambda x: x.strip('top_'))
topk_long_df['variable'] = topk_long_df['variable'].cat.remove_unused_categories()

# Reduce to median.
topk_long_df = topk_long_df.groupby(['size', 'rq', 'label', 'test', 'type', 'variable'], as_index=False)['value'].median().dropna()

display(topk_long_df)

We define a function to plot the trend.

In [ ]:
def plot_topk_trend(dataframe, size, title='', precision=0.05, y_limits=(0, 1), x_margin=0.02, y_margin=0.02, figure_size=(4, 3)):
    return (
        # Define the plot.
        p9.ggplot(
            dataframe.query('size == @size'),
            p9.aes(x='variable', y='value', group='label', color='label', shape='label'),
        )
        # Add a title.
        + p9.ggtitle(title)
        # Add the points and lines.
        + p9.geom_point(size=3, position=p9.position_dodge(width=0.2))
        + p9.geom_line(size=1, position=p9.position_dodge(width=0.2))
        # Rename the x axis and give some space to left and right.
        + p9.scale_x_discrete(name='k', expand=(0, x_margin))
        # Rename the y axis, and print the tick labels with 2 decimal digits.
        + p9.scale_y_continuous(name='Top-k Accuracy', breaks=np.linspace(0, 1, num=int(1 / precision) + 1), limits=y_limits, labels=lambda l: ['{:.2f}'.format(x) for x in l], expand=(0, y_margin))
        # Replace the names in the legend.
        + p9.scale_shape_manual(PLOT_SHAPES, name='Experiment', labels=lambda l: [experiment_label_renamer(x) for x in l])
        # Define the colors palettes.
        + p9.scale_color_manual(TOPK_MODELS_PALETTE, name='Experiment', labels=lambda l: [experiment_label_renamer(x) for x in l])
        # Define the theme for the plot.
        + p9.theme_bw()
        + p9.theme(
            # PRemove legend title, and reduce the margin.
            legend_title=p9.element_blank(),
            legend_margin=-10,
            # Set the size for the figure.
            figure_size=figure_size,
        )
    )

Then, we print all the plots, divided by size and test set.

#### Top-k trend (1k)

In [ ]:
for test_value in TEST_VALUES_ORDER:
    temp_topk_long_df = topk_long_df[(topk_long_df['size'] == '1k') & (topk_long_df['test'] == test_value)].copy()

    temp_top_models_per_rq = temp_topk_long_df.query('variable == "10"').sort_values('value', ascending=False).groupby('type')['label'].head(5)
    
    temp_topk_long_df = temp_topk_long_df[temp_topk_long_df['label'].isin(temp_top_models_per_rq)]
    temp_topk_long_df['label'] = temp_topk_long_df['label'].cat.remove_unused_categories()
    temp_topk_long_df['variable'] = temp_topk_long_df['variable'].cat.remove_unused_categories()
    temp_topk_trend_plot = plot_topk_trend(
        temp_topk_long_df,
        size='1k',
        title=f'{TEST_VALUES_LABELS[test_value]} ({SIZES_LABELS["1k"]})',
        precision=0.1,
        y_limits=(0, 0.7),
        x_margin=0.2,
        y_margin=0.03,
        figure_size=(10, 6),
    )
    display(temp_topk_trend_plot)

#### Top-k trend (Full)

In [ ]:
for test_value in TEST_VALUES_ORDER:
    temp_topk_long_df = topk_long_df[(topk_long_df['size'] == 'full') & (topk_long_df['test'] == test_value)].copy()

    temp_top_models_per_rq = temp_topk_long_df.query('variable == "10"').sort_values('value', ascending=False).groupby('type')['label'].head(3)

    temp_topk_long_df = temp_topk_long_df[temp_topk_long_df['label'].isin(temp_top_models_per_rq)]
    temp_topk_long_df['label'] = temp_topk_long_df['label'].cat.remove_unused_categories()
    temp_topk_long_df['variable'] = temp_topk_long_df['variable'].cat.remove_unused_categories()
    temp_topk_trend_plot = plot_topk_trend(
        temp_topk_long_df,
        size='full',
        title=f'{TEST_VALUES_LABELS[test_value]} ({SIZES_LABELS["full"]})',
        precision=0.05,
        y_limits=(0, 0.45),
        x_margin=0.2,
        y_margin=0.01,
        figure_size=(10, 6),
    )
    display(temp_topk_trend_plot)


### Statistical analysis

We apply the *Kruskal-Wallis Test*, which is the non-parametric version of the *ANOVA* test.
We test the median values with multiple samples of observations.

> *Null hypothesis*: the population median values of all the groups are equal.
>
> `p-value >= 0.01`: accept the null hypothesis, equal (red)
>
> `p-value < 0.01`: reject the null hypothesis, not equal (green)

Thus, we build and display the dataframe with the *p-values* for the test.

In [ ]:
# Create the dataframe.
kruskal_test_df = multiple_groups_test_dataframe(
    df,
    group_1=['size', 'test'],
    group_2='label',
    metrics=METRICS_ORDER,
    testfunc=stats.kruskal,
    check_identical=True,
)

# Pivot the dataframe for better visualization.
kruskal_test_df = (
    kruskal_test_df.pivot_table(index=['size', 'test'], columns=['metric'], values=['pvalue'])
    # Sorting.
    .reindex(SIZES_ORDER, level=0)
    .reindex(TEST_VALUES_ORDER, level=1)
    .reindex(METRICS_ORDER, axis=1, level=1)
)

display(kruskal_test_df
# Rename the values.
.rename(index=SIZES_LABELS, level=0)
.rename(index=TEST_VALUES_LABELS, level=1)
.rename(columns=METRICS_LABELS, level=1)
# Show the p-values with reduced decimal digits.
.style.format('{:.4f}')
# Apply the color filtering.
.applymap(lambda x: f'background-color: {GREEN_COLOR_HEX}' if x < SIGNIFICANCE_LEVEL else f'background-color: {RED_COLOR_HEX}')
)

We can notice that all the median values of the groups are statistically different.
Thus, we can apply a pairwise post-hoc test to understand for which couples this is valid.

We apply the *Dunn's Test*.

> *Null hypothesis*: there is no difference between the two compared groups.
>
> `p-value >= 0.01`: accept the null hypothesis, equal (green)
>
> `p-value < 0.01`: reject the null hypothesis, not equal (red)

In [ ]:
dunn_test_dfs = {}

for metric in ['mrr', 'top_10', 'aroma']:
    for size in SIZES_ORDER:
        for test_value in TEST_VALUES_ORDER:
            # Create the dataframe.
            dunn_test_df = pairwise_multiple_groups_posthoc_test_dataframe(
                df[['size', 'fold', 'test', 'label', metric]].query('size == @size and test == @test_value'),
                group_1=['size', 'test'],
                group_2='label',
                metrics=[metric],
                testfunc=sp.posthoc_dunn,
            )

            selected_experiments = dunn_test_df['label'].unique()

            # Pivot the dataframe for better visualization.
            dunn_test_df = (
                dunn_test_df.pivot_table(index='label', columns=['comparison'], values=['pvalue'])
                # Sorting
                .reindex([x for x in EXPERIMENTS_ORDER if x in selected_experiments])
                .reindex([x for x in EXPERIMENTS_ORDER if x in selected_experiments], axis=1, level=1)
            )

            # Store the dataframe.
            dunn_test_dfs[(metric, size, test_value)] = dunn_test_df

            # Print the dataframe showing the acceptance of the alternative hypothesis as green, and reject as red.
            print(f'metric={metric}, size={size}, test_value={test_value}')
            display(dunn_test_df
            # Rename the values.
            .rename(index=experiment_label_renamer, level=0)
            .rename(columns=experiment_label_renamer, level=1)
            # Show the p-values with reduced decimal digits.
            .style.format('{:.4f}')
            # Apply the color filtering.
            .applymap(lambda x: '' if x < 0 or x == 1.0 else (f'background-color: {GREEN_COLOR_HEX}' if x < SIGNIFICANCE_LEVEL else f'background-color: {RED_COLOR_HEX}'))
            )

### Effect size test

We are now interested in computing the the effect size of the statistical differences.

To do so, we use the *Vargha-Delaney A* test.

> `A = 0.5`: the group `x` performs equal to the group `y`
>
> `A < 0.5`: the group `x` performs worse than the group `y`
>
> `A > 0.5`: the group `x` performs better than the group `y`

In [ ]:
vda_test_dfs = {}

for metric in ['mrr', 'top_10', 'aroma']:
    for size in SIZES_ORDER:
        for test_value in TEST_VALUES_ORDER:
            # Create the dataframe.
            vda_test_df = pairwise_multiple_groups_vda_dataframe(
                df[['size', 'fold', 'test', 'label', metric]].query('size == @size and test == @test_value'),
                group_1=['size', 'test'],
                group_2='label',
                metrics=[metric],
            )

            selected_experiments = vda_test_df['label'].unique()

            # Pivot the dataframe for better visualization.
            vda_test_df = (
                vda_test_df.pivot_table(index='label', columns=['comparison'], values=['a', 'magnitude'], aggfunc='first')
                # Use "a" and "magnitude" as an index.
                .stack(level=0)
                # Sorting
                .reindex([x for x in EXPERIMENTS_ORDER if x in selected_experiments], level=0)
                .reindex([x for x in EXPERIMENTS_ORDER if x in selected_experiments], axis=1)
            )

            # Store the dataframe.
            vda_test_dfs[(metric, size, test_value)] = vda_test_df

            # Print the dataframe showing the colored magnitude levels.
            print(f'metric={metric}, size={size}, test_value={test_value}')
            display(vda_test_df
            # Rename the values.
            .rename(index=experiment_label_renamer, level=0)
            .rename(columns=experiment_label_renamer)
            # Show all the magnitude values as the shortest version.
            .style.format(lambda x: MAGNITUDE_LABELS[x] if isinstance(x, str) else '{:.4f}'.format(x))
            # Apply the color filtering.
            .applymap(lambda x: f'background-color: {MAGNITUDE_PALETTE[x]}' if x in MAGNITUDE_PALETTE else '')
            )